In [1]:
# Install Libraries 
!pip install -q peft

In [2]:
# Import Necessary Libraries

from peft import LoraConfig , PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer ,pipeline
import torch

2024-03-18 14:12:57.148639: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 14:12:57.148741: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 14:12:57.284109: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Attach Hugging face Token
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ["HF_TOKEN"] = user_secrets.get_secret("HF_TOKEN")

In [4]:
# Install Libraries
base_model_name = "google/gemma-2b-it"  
adapter_model_name = "Ajayk/indian-history-gemma-instructuion-finetuned"

In [5]:
#Load Base Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [7]:
# Load Peft model and tokenizer
model = PeftModel.from_pretrained(base_model, adapter_model_name)
tokenizer = AutoTokenizer.from_pretrained(base_model_name,trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" 

adapter_config.json:   0%|          | 0.00/673 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/14.3M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

In [8]:
# Merge both Peft model and Base model adapters
model = model.merge_and_unload()
model.save_pretrained("merged_adapters")

In [9]:
input_text = "What are vedhas?"
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
print(input_ids)
outputs = model.generate(**input_ids, max_length=128)
print(tokenizer.decode(outputs[0]))

{'input_ids': tensor([[     2,   1841,    708,  13191,   3212, 235336]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]], device='cuda:0')}
<bos>What are vedhas?<end_of_turn>
<start_of_turn>model
Vedhas are the early texts of the Vedic people, containing prayers, hymns, and philosophical discussions.<end_of_turn>
<eos>


### Using Pipeline Process

In [10]:
# Create the text generation pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer,device_map="auto")

In [11]:
# Generate text
generated_text = pipe("what are vedas?", max_new_tokens=100)[0]["generated_text"]
print(generated_text)

what are vedas?
model
Vedas are texts containing philosophical discussions, known as Upanishads, in the context of Vedic literature.

